# Installating R

In [ ]:
import os
import sys
import io
import re

In [ ]:
import urllib.request as request  # Python 3

In [ ]:
# get latest stable portable release download link 
# the vorboss server (UK) does not redirect back to sourceforge
base_url = "https://vorboss.dl.sourceforge.net/project/rportable/R-Portable/"
g = request.urlopen(base_url)
s = g.read().decode()
g.close;

R_version = re.findall(r'<a href="(\d\.\d\.\d)/">',s)[-1]
print("R Version: "+R_version)
paflink = base_url+R_version+"/R-Portable_"+R_version+".paf.exe"
R_installer_fullpath = os.path.join(os.environ["WINPYDIRBASE"], "t", "R.paf.exe")

In [ ]:
# start download
g = request.urlopen(paflink) 
with io.open(R_installer_fullpath, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
# checking it's there
assert os.path.isfile(R_installer_fullpath)

### download 7zip to extract R.paf.exe

In [ ]:
# first download version 9.20, since it is availabe as zip
sevenzip_dl_url = "https://vorboss.dl.sourceforge.net/project/sevenzip/7-Zip/9.20/7za920.zip"
sevenzip_file = os.path.join(os.environ["WINPYDIRBASE"], "t", "7za920.zip")
g = request.urlopen(sevenzip_dl_url) 
with io.open(sevenzip_file, 'wb') as f:
    f.write(g.read())
g.close
g = None

import zipfile
try:
    with zipfile.ZipFile(sevenzip_file) as z:
        sevenzip_install_dir = os.path.join(os.environ["WINPYDIRBASE"], "t", "7za920")
        os.mkdir(sevenzip_install_dir)
        z.extractall(sevenzip_install_dir)
        print("Extracted 7zip v9.20")
except:
    print("Invalid file")

os.remove(sevenzip_file)
sevenzip_exe = os.path.join(sevenzip_install_dir,"7za.exe")

In [ ]:
# then download version 19.00
sevenzip_dl_url = "https://vorboss.dl.sourceforge.net/project/sevenzip/7-Zip/19.00/7z1900.exe"
sevenzip_file = os.path.join(os.environ["WINPYDIRBASE"], "t", "7z1900.exe")
g = request.urlopen(sevenzip_dl_url) 
with io.open(sevenzip_file, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
# and extract 7zip v19.00  with 7zip v9.20
from subprocess import run
extr_path = os.path.join(os.environ["WINPYDIRBASE"], "t", "7z")
proc = run(sevenzip_exe+' x '+ sevenzip_file + ' -o' + extr_path)
assert proc.returncode == 0

# remove old 7zip version and 7z archive
import shutil
shutil.rmtree(sevenzip_install_dir)
os.remove(sevenzip_file)

sevenzip_exe=os.path.join(extr_path,"7z.exe")

In [ ]:
# extract R.paf.exe
extr_path = os.path.join(os.environ["WINPYDIRBASE"], "t", "R-Portable")
proc = run(sevenzip_exe+' x '+ R_installer_fullpath + ' -x!$* -o' + extr_path) # do not extract the $PLUGINSDIR folder
assert proc.returncode == 0

### write a R script to install necessary packages

In [ ]:
Rscript_text = r"""
install.packages(c('stringi'),repos="https://cloud.r-project.org/",type="binary")
install.packages(c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest'),repos="https://cloud.r-project.org/",type="binary")
devtools::install_github('IRkernel/IRkernel')
IRkernel::installspec()
"""

with io.open("temp1.R", 'w', encoding = sys.getdefaultencoding() ) as f:
    for line in Rscript_text.splitlines():
        f.write('%s\n' %  line  )

### call R script
see status in the Windows terminal window

In [ ]:
Rscript_exe = os.path.join(os.environ["WINPYDIRBASE"], "t", "R-Portable", "App", "R-Portable", "bin", "Rscript.exe")
run(Rscript_exe+" temp1.R")

#### Fix the kernel.json to allow arbitrary drive letters

the path to R is hardcoded in the kernel.json file. 
this will cause trouble, if the drive letter of the usb drive changes. 
use relative paths instead.

In [ ]:
kernel_path = os.path.join(os.environ["WINPYDIRBASE"], "settings", "kernels", "ir", "kernel.json")
assert os.path.isfile(kernel_path)

with open(kernel_path, 'r') as f:
    kernel_str = f.read()

new_kernel_str = kernel_str.replace(os.environ["WINPYDIRBASE"].replace("\\","/"),"{prefix}/..")
print(new_kernel_str)

with io.open(kernel_path, 'w', encoding = sys.getdefaultencoding() ) as f:
    for line in new_kernel_str.splitlines():
        f.write('%s\n' %  line  )

#### delete setup files

In [ ]:
os.remove(R_installer_fullpath)
os.remove("temp1.R")